# Análise dos dados com status de erros

In [39]:
import pandas as pd

df = pd.read_csv('traefik.csv',sep=',')
df = df[df['status'] >= '400']

### Existe algum padrão de horário?

In [41]:
df['data1'] = pd.to_datetime(df.data1, format='%Y-%m-%d %H:%M:%S')

In [42]:
hours = pd.DataFrame()
hours = pd.DataFrame(columns=['Horário', 'Qnt. de Acessos'])

In [43]:
for i in range(24):
    hours.loc[i]=[ i, df.data1[df['data1'].dt.hour == i].count()]

In [44]:
hours

,Horário,Qnt. de Acessos
0,0,729
1,1,684
2,2,594
3,3,651
4,4,563
5,5,574
6,6,705
7,7,815
8,8,1453
9,9,786


In [45]:
dateAcess = df.groupby([df['data1'].dt.month, df['data1'].dt.year, ]).count().sort_index()
dateAcess.rename(columns={'status': 'Acessos'}, inplace=True)
dateAcess = dateAcess.Acessos
dateAcess = dateAcess.rename_axis(['Mês', 'Ano']).reset_index()
dateAcess

,Mês,Ano,Acessos
0,1,2022,1460
1,2,2022,1341
2,3,2022,1902
3,4,2022,3208
4,5,2022,3093
5,6,2022,6870
6,7,2022,1963
7,8,2022,1420
8,9,2021,388
9,10,2021,525


### Algum IP específico?

In [46]:
ips = df.groupby(['ip']).count()
ips.rename(columns={'status': 'acessos'}, inplace=True)
ips = ips.acessos

In [47]:
ips.sort_values(ascending=False)

ip
200.129.13.122     4396
20.114.59.81       1503
200.129.47.254      948
185.7.214.104       942
187.78.38.193       845
                   ... 
137.184.215.154       1
189.48.94.14          1
137.184.34.28         1
189.2.6.114           1
185.107.56.26         1
Name: acessos, Length: 2855, dtype: int64

### Padrões de recursos?

In [48]:
recurso = df.groupby(["recurso"]).count()
recurso.rename(columns={'status': 'acessos'}, inplace=True)
recurso = recurso.acessos.reset_index()

In [49]:
recurso.sort_values(by='acessos', ascending=False)

,recurso,acessos
1,/,6308
0,-,1558
4241,/vendor/phpunit/phpunit/src/Util/PHP/eval-stdi...,710
1070,/avaliacao/tmp/assets/eff9350a/plugins/balloon...,570
28,/.env,415
...,...,...
2359,/glpi/plugins/barcode/front/send.php?file=../....,1
2360,/glpi/scripts/unlock_tasks.php?cycle=1%20UNION...,1
3315,/pesquisa/discente/minhaDeclaracao?token=7xFSv...,1
2361,/go/add-on/business-continuity/api/plugin?fold...,1


In [54]:
dbrecurso = pd.read_csv('recurso.csv', sep=',')
dbrecurso = dbrecurso[dbrecurso['status'] >= '400']

In [55]:
qtdrecursos = dbrecurso.groupby(['status', 'recurso']).count()
qtdrecursos.rename(columns={'data1': 'acessos'}, inplace=True)
qtdrecursos = qtdrecursos.acessos

In [56]:
qtdrecursos.sort_values(ascending=False)

status  recurso                                                       
404     -                                                                 1558
429     computacao                                                         852
404     js                                                                 791
        avaliacao                                                          755
        vendor                                                             713
                                                                          ... 
        go                                                                   1
        nette.micro                                                          1
        gsearch.php.en?prod=';prompt`document.domain`;                       1
        admingui                                                             1
        %04%D7%7F%BF%18%D8%7F%BF%18%D8%7F%BF%08%B7%06%08;%7Bwget,http:       1
Name: acessos, Length: 1581, dtype: int64

### Quantos métodos diferentes?

In [50]:
metodos = df.groupby(['status', 'metodo']).count()
metodos.rename(columns={'data1': 'acessos'}, inplace=True)
metodos = metodos.acessos
metodos = metodos.reset_index()
metodos.sort_values(by='acessos', ascending=False)

,status,metodo,acessos
1,404,GET,16461
2,404,HEAD,3288
5,404,POST,1735
0,404,CONNECT,1558
7,429,GET,858
3,404,OPTIONS,44
9,429,POST,26
6,404,PUT,20
8,429,HEAD,10
4,404,PATCH,1


## Gráficos

In [51]:
import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)

### Acessos por data:

In [52]:
graf2021 = go.Scatter(x=['Setembro', 'Outubro', 'Dezembro'],
                    y=dateAcess['Acessos'][8:],
                    name='2021')

graf2022 = go.Scatter(x=['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto'],
                    y=dateAcess['Acessos'][0:8],
                    name='2022')

layout = go.Layout(title='Acessos por Data',
                   yaxis={'title':'Quantidade de Acessos'},
                   xaxis={'title': 'Mês de Acesso'})

fig = go.Figure(data=[graf2021, graf2022], layout=layout)

py.iplot(fig)

### Acessos por método:

In [53]:
grafMetodo = go.Bar(x = metodos['metodo'],
                  y = metodos['acessos'])

layout = go.Layout(title='Acessos por Métodos',
                   yaxis={'title':'Quantidade de Acessos'},
                   xaxis={'title': 'Métodos'})

fig = go.Figure(data=grafMetodo, layout=layout)

py.iplot(fig)